In [ ]:
import pdfplumber
import csv

In [ ]:
pdf3 = pdfplumber.open("angrybank.pdf")

In [ ]:
import re, json
from collections import defaultdict

with open('mttq_agri_caobang.csv', 'w', newline='') as csvfile:
    fieldnames = ['transaction_code', 'amount', 'transaction_detail',
                    # 'test'
                 ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    i = 0
    for page in pdf3.pages:
        # print(page.page_number)
        start = False
        info = defaultdict(str)
        transactions = []
        
        date = None

        raw_transactions = page.extract_table()
        raw_transactions = raw_transactions[1:]
        # print(raw_transactions)
        # break
        for record in raw_transactions:
            try:
                # print(len(record), record)
                date, transaction_detail, out, _in, balance, ref_no = record
                # print(_id, timestamp, transaction_detail, amount, transaction_code)
                if not _in:
                    continue
                if '|CASZLP|' in transaction_detail:
                    # print('CASZLP', len(record), record)
                    date, transaction_detail, transaction_detail_part2, transaction_detail_part3_and_in, balance, ref_no = record
                    transaction_detal_parts = transaction_detail.split('\n')
                    transaction_detail_part3, _in = transaction_detail_part3_and_in.split(' ')
                    print(_in)
                    transaction_detail = f'{transaction_detal_parts[0]}{transaction_detail_part2}{transaction_detail_part3} {transaction_detal_parts[1]}'
                    amount = int(_in.replace(',', ''))
                    # print(amount)
                    transactions.append({
                        'transaction_detail': f'{date} {transaction_detail}'.replace('\n', ' '),
                        'amount': amount,
                        'transaction_code': ref_no,
                        # 'id': _id,
                        # 'timestamp': timestamp
                    })
                else:
                    amount = int(_in.replace(',', ''))
                    transactions.append({
                        'transaction_detail': f'{date} {transaction_detail}'.replace('\n', ' '),
                        'amount': amount,
                        'transaction_code': ref_no,
                        # 'id': _id,
                        # 'timestamp': timestamp
                    })
                # print(_id)
                # assert(int(_id) == i + 1)
                # i += 1
            except Exception as e:
                    # print(transactions[-1])
                # elif 'Foreign\nWorker\nSalary' in transaction_detail:
                #     print(len(record), record)
                # else:
                #     print(transaction_detail)
                print(len(record), record)
                pass

        # print(page, len(transactions))
        for row in transactions:
            writer.writerow(row)